In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

# SA-Co Gold/Silver 評価 (SA-Co Gold/Silver Evaluation)

このノートブックでは、SAM 3モデルの予測性能を **SA-Co (Segment Anything in Context)** データセットの **Gold** および **Silver** サブセットに対して評価する方法を解説します。

評価には `CGF1Evaluator` を使用し、セグメンテーション（segmentation）とバウンディングボックス（bounding box）の両方のタスクについて、cgF1、IoU、Recallなどの指標を計算します。

**前提条件:**
1. 評価対象のデータセット（Gold/Silver）のGround Truth（正解）アノテーションファイルが必要です。
2. モデルによる推論結果（予測ファイル）が生成されている必要があります。

## Google Colab セットアップ

Google Colabで実行する場合は、以下のセルを実行してください。
これにより、必要なライブラリのインストールと、データの読み込み準備が行われます。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        # Numpy 2.0との互換性問題を回避するためにバージョンを固定
        !pip install -q "numpy<2.0"
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

In [ ]:
# Google Colab環境のセットアップ
try:
    import google.colab
    IN_COLAB = True
    print("Google Colab環境で実行中")
except ImportError:
    IN_COLAB = False
    print("ローカル環境で実行中")

if IN_COLAB:
    # 1. Google Driveをマウント (推奨)
    # 自分のデータやコードを使用する場合は、Google Driveにアップロードしてマウントします
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 2. SAM3のインストールとパス設定
    import os
    import sys
    
    # Google Drive内のSAM3ディレクトリのパス
    # ※ご自身の環境に合わせてパスを変更してください
    DRIVE_SAM3_PATH = "/content/drive/MyDrive/sam3"
    
    if os.path.exists(DRIVE_SAM3_PATH):
        print(f"Google Drive内のSAM3が見つかりました: {DRIVE_SAM3_PATH}")
        os.chdir(DRIVE_SAM3_PATH)
        print("カレントディレクトリを変更しました。")
        
        # 依存関係のインストール
        print("依存関係をインストールしています...")
        !pip install -q -e .
        !pip install -q pycocotools
        
    else:
        print(f"Google Drive内に {DRIVE_SAM3_PATH} が見つかりませんでした。")
        print("GitHubからSAM3をクローンしてインストールします...")
        
        # GitHubからクローン
        if not os.path.exists("/content/sam3"):
            !git clone https://github.com/facebookresearch/sam3.git /content/sam3
            
        os.chdir("/content/sam3")
        print("カレントディレクトリを /content/sam3 に変更しました。")
        
        # 依存関係のインストール
        !pip install -q -e .
        !pip install -q pycocotools
    
    print("セットアップが完了しました。")
    print(f"現在の作業ディレクトリ: {os.getcwd()}")

### データの使用について

- **Google Driveを使用する場合**: `GT_DIR` や `PRED_DIR` には `/content/drive/MyDrive/...` から始まるパスを指定してください。
- **GitHubからクローンした場合**: 左側のファイルブラウザから `/content/sam3` 内を確認できます。データは別途アップロードが必要です。

In [5]:
import copy
import json
import os

import numpy as np

from pycocotools.coco import COCO
from sam3.eval.cgf1_eval import CGF1Evaluator

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:287: UserWarning: 
NVIDIA GeForce RTX 5060 Ti with CUDA capability sm_120 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_50 sm_60 sm_70 sm_75 sm_80 sm_86 sm_90.
If you want to use the NVIDIA GeForce RTX 5060 Ti GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(


# SA-Co/Gold データセット

### パスの設定

以下のセルで、Ground Truth (GT) アノテーションファイルがあるディレクトリ (`GT_DIR`) と、予測ファイルがあるディレクトリ (`PRED_DIR`) を指定してください。

**Google Colabの場合の例:**
- Google Driveを使用: `GT_DIR = "/content/drive/MyDrive/saco_data/annotations"`
- アップロードしたファイル: `GT_DIR = "/content/annotations"`

In [ ]:
# GTアノテーションと予測ファイルが存在するディレクトリに更新してください
# ローカル環境の例: GT_DIR = "/path/to/annotations"
# Colab + Google Driveの例: GT_DIR = "/content/drive/MyDrive/saco_data/annotations"
GT_DIR = ""  # ここにパスを記入
PRED_DIR = ""  # ここにパスを記入

In [ ]:
# 7つのSA-Co/Goldサブセットに対するGTファイルの相対ファイル名
saco_gold_gts = {
    # MetaCLIP キャプション生成
    "metaclip_nps": [
            "gold_metaclip_merged_a_release_test.json",
            "gold_metaclip_merged_b_release_test.json",
            "gold_metaclip_merged_c_release_test.json",
    ],
    # SA-1B キャプション生成
    "sa1b_nps": [
            "gold_sa1b_merged_a_release_test.json",
            "gold_sa1b_merged_b_release_test.json",
            "gold_sa1b_merged_c_release_test.json",
    ],
    # Crowded (密集)
    "crowded": [
            "gold_crowded_merged_a_release_test.json",
            "gold_crowded_merged_b_release_test.json",
            "gold_crowded_merged_c_release_test.json",
    ],
    # FG Food (食品)
    "fg_food": [
            "gold_fg_food_merged_a_release_test.json",
            "gold_fg_food_merged_b_release_test.json",
            "gold_fg_food_merged_c_release_test.json",
    ],
    # FG Sports (スポーツ用品)
    "fg_sports_equipment": [
            "gold_fg_sports_equipment_merged_a_release_test.json",
            "gold_fg_sports_equipment_merged_b_release_test.json",
            "gold_fg_sports_equipment_merged_c_release_test.json",
    ],
    # Attributes (属性)
    "attributes": [
            "gold_attributes_merged_a_release_test.json",
            "gold_attributes_merged_b_release_test.json",
            "gold_attributes_merged_c_release_test.json",
    ],
    # Wiki common
    "wiki_common": [
            "gold_wiki_common_merged_a_release_test.json",
            "gold_wiki_common_merged_b_release_test.json",
            "gold_wiki_common_merged_c_release_test.json",
    ],
}

## 全7つのSA-Co/Goldサブセットに対するオフライン評価の実行

全7つのデータセットに対して推論が既に実行されていることを前提としています。デフォルトの設定では、予測結果は予測可能なフォルダに出力されます。

In [ ]:
results_gold = {}
results_gold_bbox = {}

for subset_name, gts in saco_gold_gts.items():
    print("Processing subset: ", subset_name)
    gt_paths = [os.path.join(GT_DIR, gt) for gt in gts]
    pred_path = os.path.join(PRED_DIR, f"gold_{subset_name}/dumps/gold_{subset_name}/coco_predictions_segm.json")
    
    evaluator = CGF1Evaluator(gt_path=gt_paths, verbose=True, iou_type="segm") 
    summary = evaluator.evaluate(pred_path)
    print(summary)

    cur_results = {}
    cur_results["cgf1"] = summary["cgF1_eval_segm_cgF1"] * 100
    cur_results["il_mcc"] = summary["cgF1_eval_segm_IL_MCC"]
    cur_results["pmf1"] = summary["cgF1_eval_segm_positive_micro_F1"] * 100
    results_gold[subset_name] = cur_results

    # バウンディングボックスも評価    
    evaluator = CGF1Evaluator(gt_path=gt_paths, verbose=True, iou_type="bbox") 
    summary = evaluator.evaluate(pred_path)
    print(summary)

    cur_results = {}
    cur_results["cgf1"] = summary["cgF1_eval_bbox_cgF1"] * 100
    cur_results["il_mcc"] = summary["cgF1_eval_bbox_IL_MCC"]
    cur_results["pmf1"] = summary["cgF1_eval_bbox_positive_micro_F1"] * 100
    results_gold_bbox[subset_name] = cur_results

In [ ]:
# 平均を計算
METRICS = ["cgf1", "il_mcc", "pmf1"]
avg_stats, avg_stats_bbox = {}, {}
for key in METRICS:
    avg_stats[key] = sum(res[key] for res in results_gold.values()) / len(results_gold)
    avg_stats_bbox[key] = sum(res[key] for res in results_gold_bbox.values()) / len(results_gold_bbox)
results_gold["Average"] = avg_stats
results_gold_bbox["Average"] = avg_stats_bbox

In [ ]:
# セグメンテーション結果を整形して表示
from IPython.display import HTML, display

row1, row2, row3 = "", "", ""
for subset in results_gold:
    row1 += f'<th colspan="3" style="text-align:center;border-left-style:solid;border-left-width:1px">{subset}</th>'
    row2 += "<th style='border-left-style:solid;border-left-width:1px'>" + "</th><th>".join(METRICS) + "</th>"
    row3 += "<td style='border-left-style:solid;border-left-width:1px'>" + "</td><td>".join([str(round(results_gold[subset][k], 2)) for k in METRICS])  + "</td>"

display(HTML(
   f"<table><thead><tr>{row1}</tr><tr>{row2}</tr></thead><tbody><tr>{row3}</tr></tbody></table>"
))

In [ ]:
# バウンディングボックス検出結果を整形して表示
from IPython.display import HTML, display

row1, row2, row3 = "", "", ""
for subset in results_gold:
    row1 += f'<th colspan="3" style="text-align:center;border-left-style:solid;border-left-width:1px">{subset}</th>'
    row2 += "<th style='border-left-style:solid;border-left-width:1px'>" + "</th><th>".join(METRICS) + "</th>"
    row3 += "<td style='border-left-style:solid;border-left-width:1px'>" + "</td><td>".join([str(round(results_gold_bbox[subset][k], 2)) for k in METRICS])  + "</td>"

display(HTML(
   f"<table><thead><tr>{row1}</tr><tr>{row2}</tr></thead><tbody><tr>{row3}</tr></tbody></table>"
))

# SA-Co/Silver データセット

In [ ]:
# GTアノテーションと予測ファイルが存在するディレクトリに更新してください
GT_DIR = ""  # ここにパスを記入
PRED_DIR = ""  # ここにパスを記入

In [ ]:
saco_silver_gts = {
    "bdd100k": "silver_bdd100k_merged_test.json",
    "droid": "silver_droid_merged_test.json",
    "ego4d": "silver_ego4d_merged_test.json",
    "food_rec": "silver_food_rec_merged_test.json",
    "geode": "silver_geode_merged_test.json",
    "inaturalist": "silver_inaturalist_merged_test.json",
    "nga_art": "silver_nga_art_merged_test.json",
    "sav": "silver_sav_merged_test.json",
    "yt1b": "silver_yt1b_merged_test.json",
    "fathomnet": "silver_fathomnet_test.json",
}

In [ ]:
results_silver = {}
results_silver_bbox = {}

for subset_name, gt in saco_silver_gts.items():
    print("Processing subset: ", subset_name)
    gt_path = os.path.join(GT_DIR, gt)
    pred_path = os.path.join(PRED_DIR, f"silver_{subset_name}/dumps/silver_{subset_name}/coco_predictions_segm.json")
    
    evaluator = CGF1Evaluator(gt_path=gt_path, verbose=True, iou_type="segm") 
    summary = evaluator.evaluate(pred_path)
    print(summary)

    cur_results = {}
    cur_results["cgf1"] = summary["cgF1_eval_segm_cgF1"] * 100
    cur_results["il_mcc"] = summary["cgF1_eval_segm_IL_MCC"]
    cur_results["pmf1"] = summary["cgF1_eval_segm_positive_micro_F1"] * 100
    results_silver[subset_name] = cur_results

    # バウンディングボックスも評価    
    evaluator = CGF1Evaluator(gt_path=gt_path, verbose=True, iou_type="bbox") 
    summary = evaluator.evaluate(pred_path)
    print(summary)

    cur_results = {}
    cur_results["cgf1"] = summary["cgF1_eval_bbox_cgF1"] * 100
    cur_results["il_mcc"] = summary["cgF1_eval_bbox_IL_MCC"]
    cur_results["pmf1"] = summary["cgF1_eval_bbox_positive_micro_F1"] * 100
    results_silver_bbox[subset_name] = cur_results


In [ ]:
# 平均を計算
METRICS = ["cgf1", "il_mcc", "pmf1"]
avg_stats, avg_stats_bbox = {}, {}
for key in METRICS:
    avg_stats[key] = sum(res[key] for res in results_silver.values()) / len(results_silver)
    avg_stats_bbox[key] = sum(res[key] for res in results_silver_bbox.values()) / len(results_silver_bbox)
results_silver["Average"] = avg_stats
results_silver_bbox["Average"] = avg_stats_bbox

In [ ]:
# セグメンテーション結果を整形して表示
from IPython.display import HTML, display

row1, row2, row3 = "", "", ""
for subset in results_silver:
    row1 += f'<th colspan="3" style="text-align:center;border-left-style:solid;border-left-width:1px">{subset}</th>'
    row2 += "<th style='border-left-style:solid;border-left-width:1px'>" + "</th><th>".join(METRICS) + "</th>"
    row3 += "<td style='border-left-style:solid;border-left-width:1px'>" + "</td><td>".join([str(round(results_silver[subset][k], 2)) for k in METRICS])  + "</td>"

display(HTML(
   f"<table><thead><tr>{row1}</tr><tr>{row2}</tr></thead><tbody><tr>{row3}</tr></tbody></table>"
))

In [ ]:
# バウンディングボックス検出結果を整形して表示
from IPython.display import HTML, display

row1, row2, row3 = "", "", ""
for subset in results_silver_bbox:
    row1 += f'<th colspan="3" style="text-align:center;border-left-style:solid;border-left-width:1px">{subset}</th>'
    row2 += "<th style='border-left-style:solid;border-left-width:1px'>" + "</th><th>".join(METRICS) + "</th>"
    row3 += "<td style='border-left-style:solid;border-left-width:1px'>" + "</td><td>".join([str(round(results_silver_bbox[subset][k], 2)) for k in METRICS])  + "</td>"

display(HTML(
   f"<table><thead><tr>{row1}</tr><tr>{row2}</tr></thead><tbody><tr>{row3}</tr></tbody></table>"
))